In [32]:
!git clone https://github.com/WebCrawlerr/GAT.git

Cloning into 'GAT'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 149 (delta 82), reused 94 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (149/149), 120.05 KiB | 6.00 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [33]:
# Pobranie zmian
!git pull origin m

fatal: couldn't find remote ref m


In [34]:
# Wyświetla listę plików w pobranym folderze
!ls -R  GAT

GAT:
gat-inz.ipynb  models		      requirements.txt	  src
main.py        prepare_leash_data.py  run_experiments.py  tests

GAT/models:
best_model.pth	fold_0_best_model.pth

GAT/src:
config.py	    dataset.py	 model.py     __pycache__  utils.py
data_processing.py  features.py  optimize.py  train.py

GAT/src/__pycache__:
config.cpython-312.pyc		 model.cpython-312.pyc
data_processing.cpython-312.pyc  optimize.cpython-312.pyc
dataset.cpython-312.pyc		 train.cpython-312.pyc
features.cpython-312.pyc	 utils.cpython-312.pyc

GAT/tests:
test_cv.py	  test_optimize.py  test_plotting.py   verify_plots.py
test_features.py  test_pipeline.py  verify_metrics.py


In [35]:
import shutil
import os
import sys
# 1. Zmiana katalogu roboczego (odpowiednik komendy 'cd')
os.chdir('./GAT')
print(f"Aktualny katalog: {os.getcwd()}")

Aktualny katalog: /kaggle/working/GAT/GAT/GAT/GAT


In [36]:
# Najpierw sprawdźmy wersję po restarcie (czasem GPU ma starszego PyTorcha)
import torch
version = torch.__version__
print(f"PyTorch version: {version}")

# Instalacja PyG i bibliotek pomocniczych (korzystamy z gotowych paczek .whl, żeby nie kompilować przez 20 min)
# UWAGA: Ta komenda dynamicznie dobiera link do wersji Twojego PyTorcha
torch_version_suffix = version.split('+')[0].replace('.', '') # np. 260 lub 240
cuda_version_suffix = 'cu' + torch.version.cuda.replace('.', '') # np. cu121

print(f"Instaluję dla: torch-{torch_version_suffix} + {cuda_version_suffix}")

# Instalacja zależności
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv \
  -f https://data.pyg.org/whl/torch-{version}.html

# Instalacja reszty z Twojego pliku (bez ponownego instalowania torch-geometric, bo zrobiliśmy to wyżej)
!pip install -r requirements.txt

PyTorch version: 2.6.0+cu124
Instaluję dla: torch-260 + cu124
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


In [37]:
import torch
import torch_geometric

print(f"CUDA dostępne: {torch.cuda.is_available()}")
print(f"Wersja PyTorch: {torch.__version__}")
print(f"Wersja PyG: {torch_geometric.__version__}")

# Próba importu kluczowych warstw
try:
    from torch_geometric.nn import GATConv
    print("✅ GATConv zaimportowany pomyślnie!")
except ImportError as e:
    print(f"❌ Błąd importu GATConv: {e}")

# Sprawdzenie czy mamy backendy (scatter/sparse)
# PyG 2.7+ potrafi czasem działać bez nich (wolniej), ale warto sprawdzić
try:
    import torch_scatter
    print("✅ torch_scatter obecny")
except ImportError:
    print("⚠️ torch_scatter BRAK (Może to spowolnić trening lub wywołać błąd)")

try:
    import torch_sparse
    print("✅ torch_sparse obecny")
except ImportError:
    print("⚠️ torch_sparse BRAK (Wymagany dla niektórych operatorów)")

CUDA dostępne: True
Wersja PyTorch: 2.6.0+cu124
Wersja PyG: 2.7.0
✅ GATConv zaimportowany pomyślnie!
✅ torch_scatter obecny
✅ torch_sparse obecny


In [38]:
#!python main.py --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed --cv 5

In [39]:
#!python main.py --optimize --n_trials 15 --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed
#--cv 5

In [40]:
#!python run_experiments.py --processed_dir ./data/processed --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv

In [41]:
#PRZYGOTOWANIE DANYCH BELKA
#!python prepare_leash_data.py --input_file /kaggle/input/leash-BELKA/train.csv --output_dir /kaggle/working/ --protein BRD4 --negative_rate 1

In [42]:
import sys
sys.path.append('/kaggle/working/GAT') # Dostosuj ścieżkę

from src.data_processing import load_and_filter_data, clean_and_label_data

# Ścieżka do danych w Kaggle
data_path = '/kaggle/input/bindingdb-smiles/BindingDB_All.tsv'


df = load_and_filter_data(data_path)
df_clean = clean_and_label_data(df)
# Funkcja clean_and_label_data sama wypisuje statystyki na końcu

Loading and filtering data from /kaggle/input/bindingdb-smiles/BindingDB_All.tsv in chunks of 100000...
Processed 1000000 rows...
Processed 2000000 rows...
Processed 3000000 rows...
Finished processing 3078912 rows.


In [43]:
import os
import shutil
from IPython.display import FileLink

# Nazwa pliku, którego szukamy
zip_name = 'GAT_BRD4_Results.zip'

# 1. Znajdź plik w całym systemie plików roboczych
found_path = None
for root, dirs, files in os.walk('/kaggle/working'):
    if zip_name in files:
        found_path = os.path.join(root, zip_name)
        break

if found_path:
    print(f"✅ Znaleziono plik tutaj: {found_path}")
    
    # 2. Przenieś go do głównego katalogu (Dla pewności)
    target_path = f'/kaggle/working/{zip_name}'
    
    if found_path != target_path:
        shutil.move(found_path, target_path)
        print(f"📦 Przeniesiono plik do głównego katalogu: {target_path}")
    
    # 3. Zmień katalog roboczy na główny (żeby FileLink zadziałał)
    os.chdir('/kaggle/working')
    
    # 4. Wyświetl link
    print("👇 Kliknij poniżej:")
    display(FileLink(zip_name))
else:
    print("❌ Nie znaleziono pliku ZIP. Upewnij się, że poprzednia komórka wykonała się bez błędów.")
    # Sprawdźmy co w ogóle mamy na dysku
    print("Lista plików w obecnym folderze:")
    print(os.listdir('.'))

❌ Nie znaleziono pliku ZIP. Upewnij się, że poprzednia komórka wykonała się bez błędów.
Lista plików w obecnym folderze:
['gat-inz.ipynb', '.git', 'src', '.gitignore', 'models', 'prepare_leash_data.py', 'main.py', 'requirements.txt', 'tests', 'run_experiments.py']
